In [17]:
# load data
import pandas as pd

results_file = "10-12-results.csv"
misc_file = "10-12-misc.csv"
stages_file = "10-12-stages.csv"
practice_file = "10-11-practice.csv"

results = pd.read_csv(results_file)
misc = pd.read_csv(misc_file)
stages = pd.read_csv(stages_file)
practice = pd.read_csv(practice_file)

In [18]:
# aggregation of practice data to eliminate 'duplicate' observations 
# the only race consistently having multiple practices is the Daytona500, but I think this is best practice for now
practice_agg = (
    practice.groupby(["race_id", "driver_id"])
    .agg({
        "BestLapRank" : "mean",
        "OverAllAvgRank" : "mean",
        "Con5LapRank" : "mean",
        "Con10LapRank" : "mean",
        "Con15LapRank" : "mean",
        "Con20LapRank" : "mean",
        "Con25LapRank" : "mean",
        "Con30LapRank" : "mean"
    })
)

In [19]:
# pivoting the stage dataset to eliminate 'duplicate' observations
# doing this rather than aggregation since I want to preserve stage 1 and stage 2 as seperate parts of the race (not averaged together)
stages_wide = stages.pivot_table(
    index=["race_id", "driver_id"],
    columns="stage_number",
    values=["position", "stage_points"]
)

stages_wide.columns = [
    f"stage_{col[1]}_{col[0]}" for col in stages_wide.columns.to_flat_index()
]

stages_wide = stages_wide.reset_index()

In [20]:
# data merging
df = results.copy()
df = pd.merge(df, misc, on=["race_id", "driver_id"], how="outer", suffixes=("", "_misc"))
df = pd.merge(df, stages_wide, on=["race_id", "driver_id"], how="outer")
df = pd.merge(df, practice_agg, on=["race_id", "driver_id"], how="outer")

In [21]:
# shows duplicates (same driver_id AND race_id in multiple rows)
# dupes = df[df.duplicated(subset=["race_id", "driver_id"], keep=False)]
# print(dupes.sort_values(["race_id", "driver_id"]))
# ensure no duplicates
assert df.duplicated(subset=["race_id", "driver_id"]).sum() == 0

In [22]:
import importlib, rolling_lagging
importlib.reload(rolling_lagging)
from rolling_lagging import lagging_rolling_generator, reconcile_driver_carteams

lagroll_features = [
    "finishing_position", "starting_position", # "points_position", 
    "stage_1_position", "stage_2_position", 
    "mid_ps", "closing_ps", "avg_ps", "BestLapRank", "OverAllAvgRank",
    "laps_completed", "laps_led", "points_earned", "fast_laps", "top15_laps", "rating"
]

# feature engineering, lagging & rolling averages
# df, features, sort_list, filter_list, windows_list, suffix, min_periods
# directly recent races (momentum):
# df = lagging_rolling_generator(df, lagroll_features, ["driver_id"], ["driver_id"], [3, 5, 10], "general", 3)
# df = lagging_rolling_generator(df, lagroll_features, ["team_name"], ["team_name"], [3, 5, 10], "general_team", 3)
# df = lagging_rolling_generator(df, lagroll_features, ["team_name", "car_number"], ["team_name", "car_number"], [3, 5, 10], "general_carteam", 3)
df = lagging_rolling_generator(df, lagroll_features, ["driver_id"], ["driver_id"], [5], "general", 1)
df = lagging_rolling_generator(df, lagroll_features, ["team_name"], ["team_name"], [3, 5], "general_team", 1)
df = lagging_rolling_generator(df, lagroll_features, ["team_name", "car_number"], ["team_name", "car_number"], [3, 5], "general_carteam", 1)
# most recent at track type:
df = lagging_rolling_generator(df, lagroll_features, ["driver_id"], ["driver_id", "track_type"], [5], "tracktype", 1)
df = lagging_rolling_generator(df, lagroll_features, ["team_name"], ["team_name", "track_type"], [5], "tracktype_team", 1)
df = lagging_rolling_generator(df, lagroll_features, ["team_name", "car_number"], ["team_name", "car_number", "track_type"], [5], "tracktype_carteam", 1)
# most recent at specific track:
df = lagging_rolling_generator(df, lagroll_features, ["driver_id"], ["driver_id", "track_name"], [3], "track", 1)
df = lagging_rolling_generator(df, lagroll_features, ["team_name"], ["team_name", "track_name"], [3], "track_team", 1)
df = lagging_rolling_generator(df, lagroll_features, ["team_name", "car_number"], ["team_name", "car_number", "track_name"], [3], "track_carteam", 1)

In [23]:
# fill NaNs in lagroll features with carteam or team averages
# df = reconcile_driver_carteams(df, lagroll_features, [3, 5, 10], "general")
df = reconcile_driver_carteams(df, lagroll_features, [5], "general")
df = reconcile_driver_carteams(df, lagroll_features, [5], "tracktype")
df = reconcile_driver_carteams(df, lagroll_features, [3], "track")

In [24]:
# dropping features irrelevant to benchmark model

# keeping for visibility when reviewing df as csv
keep_cols = [
    "finishing_position", "race_id", "race_season", "race_name", "track_name", "race_date",
    "driver_fullname", "driver_id", "car_number", "team_name", "car_make",
    "crew_chief_fullname", "finishing_status"
]

# remove the actual in-race data per observation but keep P&Q
in_race_leakage = [
    'finishing_position', 'laps_completed', 'laps_led', 'points_earned', 'playoff_points_earned', 
    'points_position', 'mid_ps', 'closing_ps', 'avg_ps', 'fast_laps', 'top15_laps', 'rating', 
    'stage_1_position', 'stage_2_position'
]

# keep engineered driver features for model, but not team / carteam features (those were only used to fill missing driver stats)
lagroll_cols = [
    col for col in df.columns
    if any(feat in col for feat in lagroll_features)
    and "_team" not in col
    and "_carteam" not in col
    and col not in in_race_leakage
]
print(lagroll_cols)

final_cols = keep_cols + lagroll_cols
df = df[final_cols]
# df.to_csv("reconciled_driver_teams.csv", index=False)

['starting_position', 'BestLapRank', 'OverAllAvgRank', 'finishing_position_lag1_general', 'finishing_position_roll5_general', 'starting_position_lag1_general', 'starting_position_roll5_general', 'stage_1_position_lag1_general', 'stage_1_position_roll5_general', 'stage_2_position_lag1_general', 'stage_2_position_roll5_general', 'mid_ps_lag1_general', 'mid_ps_roll5_general', 'closing_ps_lag1_general', 'closing_ps_roll5_general', 'avg_ps_lag1_general', 'avg_ps_roll5_general', 'BestLapRank_lag1_general', 'BestLapRank_roll5_general', 'OverAllAvgRank_lag1_general', 'OverAllAvgRank_roll5_general', 'laps_completed_lag1_general', 'laps_completed_roll5_general', 'laps_led_lag1_general', 'laps_led_roll5_general', 'points_earned_lag1_general', 'points_earned_roll5_general', 'fast_laps_lag1_general', 'fast_laps_roll5_general', 'top15_laps_lag1_general', 'top15_laps_roll5_general', 'rating_lag1_general', 'rating_roll5_general', 'finishing_position_lag1_tracktype', 'finishing_position_roll5_tracktype

In [25]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from scipy.stats import spearmanr
import numpy as np

# Features and target
df_train = df.dropna(subset=["finishing_position"]).copy()
X = df_train[lagroll_cols]
y = df_train["finishing_position"]
groups = df_train["race_id"]

# Define pipeline: impute -> scale -> model
pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler()),
    ("ridge", Ridge())
])

# Parameter grid for alpha
param_grid = {"ridge__alpha": [1000, 1400, 1600, 1800, 2000]}

# Grouped CV
cv = GroupKFold(n_splits=5)

# Grid search
grid = GridSearchCV(pipe, param_grid, scoring="neg_mean_squared_error", cv=cv)
grid.fit(X, y, groups=groups)

print("Best alpha:", grid.best_params_)
best_model = grid.best_estimator_

# Get coefficients
ridge_model = best_model.named_steps["ridge"]
coef = pd.Series(ridge_model.coef_, index=lagroll_cols)
print(coef.sort_values(ascending=False).head(20))

# Apply to full df
df["weighted_score_lr"] = best_model.predict(df[lagroll_cols])
df["pred_rank_lr"] = df.groupby("race_id")["weighted_score_lr"].rank(method="min", ascending=True)  
# ascending=True because lower predicted finish = better (P1)

def race_spearman(g):
    if g["weighted_score_lr"].nunique() < 2:
        return np.nan
    return spearmanr(g["weighted_score_lr"], g["finishing_position"]).correlation

race_corrs = (
    df.dropna(subset=["finishing_position"])
      .groupby("race_id")
      .apply(race_spearman)
)

print("\nSpearman mean:", race_corrs.mean().round(3))
print("Spearman median:", race_corrs.median().round(3))
print("Number of races evaluated:", race_corrs.notna().sum())

Best alpha: {'ridge__alpha': 1000}
starting_position                    1.301448
OverAllAvgRank                       0.550548
OverAllAvgRank_roll5_general         0.475205
BestLapRank                          0.455381
laps_led_roll5_tracktype             0.338009
starting_position_roll5_general      0.303888
mid_ps_roll5_tracktype               0.300909
stage_2_position_roll5_general       0.257053
mid_ps_lag1_general                  0.248870
rating_lag1_general                  0.229422
avg_ps_roll3_track                   0.228888
stage_1_position_lag1_tracktype      0.222874
avg_ps_roll5_tracktype               0.222150
OverAllAvgRank_roll3_track           0.205136
finishing_position_roll3_track       0.190681
BestLapRank_roll5_general            0.179840
avg_ps_roll5_general                 0.173963
closing_ps_lag1_general              0.157555
starting_position_roll5_tracktype    0.155986
starting_position_lag1_general       0.144770
dtype: float64

Spearman mean: 0.433
Spearman

C:\Users\joeah\AppData\Local\Temp\ipykernel_16944\2951885294.py:53: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(race_spearman)


In [26]:
race_id_to_check = 5582 

race_df = df[df["race_id"] == race_id_to_check].copy()

# compute absolute error between predicted rank and actual finish
race_df["abs_error_weighted"] = (race_df["pred_rank_lr"] - race_df["finishing_position"]).abs()

# compute summary metrics
mean_error = race_df["abs_error_weighted"].mean()
median_error = race_df["abs_error_weighted"].median()

print(f"Weighted model for race {race_id_to_check}:")
print(f"Average (mean) error: {mean_error:.2f}")
print(f"Median error: {median_error:.2f}")

# Sort by predicted rank (best at the top)
race_df = race_df.sort_values("pred_rank_lr")

# Select useful columns to display
cols_to_show = [
    "race_id",
    "driver_fullname",
    # "team_name",
    # "weighted_score_lr_flipped",
    "pred_rank_lr",
    "finishing_position"  # actual for comparison
]

print(race_df[cols_to_show].head(40))
race_df.to_csv("race_db.csv", index=False)

Weighted model for race 5582:
Average (mean) error: 9.05
Median error: 8.00
      race_id      driver_fullname  pred_rank_lr  finishing_position
5222     5582        William Byron           1.0                36.0
5208     5582          Kyle Larson           2.0                 2.0
5219     5582     Christopher Bell           3.0                 3.0
5212     5582        Tyler Reddick           4.0                 5.0
5211     5582        Chase Elliott           5.0                18.0
5195     5582         Denny Hamlin           6.0                 1.0
5207     5582        Bubba Wallace           7.0                22.0
5204     5582        Ross Chastain           8.0                23.0
5224     5582        Chase Briscoe           9.0                 4.0
5203     5582       Chris Buescher          10.0                12.0
5230     5582             Ty Gibbs          11.0                34.0
5206     5582          Ryan Blaney          12.0                38.0
5209     5582          Alex

In [27]:
racing_insights_file = "10-12-racing-insights.csv"
racing_insights = pd.read_csv(racing_insights_file)

racing_insights["race_id"] = racing_insights["race_id"].astype(df["race_id"].dtype)
racing_insights["driver_id"] = racing_insights["driver_id"].astype(df["driver_id"].dtype)

df = pd.merge(df, racing_insights, on=["race_id", "driver_id"], how="outer", suffixes=("", "_racing_insights"))

assert df.duplicated(subset=["race_id", "driver_id"]).sum() == 0

df_2025 = df[df["race_season"] == 2025].copy()

print(df_2025.columns.tolist())

['finishing_position', 'race_id', 'race_season', 'race_name', 'track_name', 'race_date', 'driver_fullname', 'driver_id', 'car_number', 'team_name', 'car_make', 'crew_chief_fullname', 'finishing_status', 'starting_position', 'BestLapRank', 'OverAllAvgRank', 'finishing_position_lag1_general', 'finishing_position_roll5_general', 'starting_position_lag1_general', 'starting_position_roll5_general', 'stage_1_position_lag1_general', 'stage_1_position_roll5_general', 'stage_2_position_lag1_general', 'stage_2_position_roll5_general', 'mid_ps_lag1_general', 'mid_ps_roll5_general', 'closing_ps_lag1_general', 'closing_ps_roll5_general', 'avg_ps_lag1_general', 'avg_ps_roll5_general', 'BestLapRank_lag1_general', 'BestLapRank_roll5_general', 'OverAllAvgRank_lag1_general', 'OverAllAvgRank_roll5_general', 'laps_completed_lag1_general', 'laps_completed_roll5_general', 'laps_led_lag1_general', 'laps_led_roll5_general', 'points_earned_lag1_general', 'points_earned_roll5_general', 'fast_laps_lag1_general',

In [28]:
from scipy.stats import spearmanr
import numpy as np
import pandas as pd

def race_spearman_corrs(g):
    result = {}
    
    # Only calculate if we have at least 2 drivers
    if g["finishing_position"].nunique() > 1:
        # Your model correlation
        if g["pred_rank_lr"].nunique() > 1:
            result["my_model_corr"] = spearmanr(g["pred_rank_lr"], g["finishing_position"]).correlation
        else:
            result["my_model_corr"] = np.nan
        
        # Racing Insights correlation
        if g["pred_finish_ri"].nunique() > 1:
            result["ri_corr"] = spearmanr(g["pred_finish_ri"], g["finishing_position"]).correlation
        else:
            result["pred_finish_ri"] = np.nan
    
    return pd.Series(result)

race_corrs_2025 = (
    df_2025.dropna(subset=["finishing_position"])
    .groupby("race_id")
    .apply(race_spearman_corrs)
)

C:\Users\joeah\AppData\Local\Temp\ipykernel_16944\2115154869.py:27: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(race_spearman_corrs)


In [29]:
print("Race-by-race Spearman Correlations (2025):")
print(race_corrs_2025.round(3))

print("\nAverage correlations across 2025:")
print("My model:", race_corrs_2025["my_model_corr"].mean().round(3))
print("Racing Insights:", race_corrs_2025["ri_corr"].mean().round(3))

Race-by-race Spearman Correlations (2025):
         my_model_corr  ri_corr
race_id                        
5546             0.315    0.208
5547             0.233    0.243
5548             0.308    0.359
5549             0.309    0.403
5550             0.689    0.721
5551             0.385    0.479
5552             0.386    0.441
5553             0.652    0.579
5554            -0.010    0.042
5555             0.173    0.301
5556             0.619    0.555
5557             0.303    0.220
5558             0.443    0.427
5563             0.255    0.285
5564             0.732    0.712
5565             0.438    0.451
5566             0.698    0.633
5567             0.630    0.691
5568             0.674    0.665
5569             0.371    0.412
5570             0.532    0.518
5571             0.451    0.413
5572             0.364    0.309
5573             0.433    0.484
5574             0.093    0.099
5575             0.518    0.472
5576            -0.030   -0.185
5577             0.564    0.5

In [30]:
df_2025["abs_error_my_model"] = (df_2025["pred_rank_lr"] - df_2025["finishing_position"]).abs()
df_2025["abs_error_ri"] = (df_2025["pred_finish_ri"] - df_2025["finishing_position"]).abs()

print("\nMean Absolute Error (lower = better):")
print("My model:", df_2025["abs_error_my_model"].mean().round(2))
print("Racing Insights:", df_2025["abs_error_ri"].mean().round(2))


Mean Absolute Error (lower = better):
My model: 9.2
Racing Insights: 9.17
